#  Unit 2.4a Answers
> Answers to 2.4A
- toc: true
- image: /images/pythonlogo.png
- categories: []


# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
    2. SQLAlchemy db object


In [93]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///files/sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app) 



# Questions
- I've seen the flask object operate in the AWS backend on my localhost in order to run a flask app and flask code.
- The SQLAlchemy class has also been seen with AWS backend, in order to protect the SQLite database and operate with it.

## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User
    - db.Model inheritance
    - _init_ method
    - ```@property```, ```@<column>.setter```
    - create, read, update, delete methods

In [94]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json
import uuid
from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''


class Animal(db.Model): # inherited, we can use what db.Model has
    # uses the methods and class variable (need to call .super())
    __tablename__ = 'monke'  

    id = db.Column(db.Integer, primary_key=True)
    _lbs_food = db.Column(db.Integer)
    _animal_type = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object
    # __init__ initializes an attribute and makes a class item
    def __init__(self, name, animal, uid, food, dob=datetime.today()):
        self._name = name    
        self._uid = uid
        self._lbs_food = food
        self._animal_type = animal
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    ###
    # getters and setters
    # Getters get, setters set
    # @ used for get. @name.set are setters
    # These are decorator functions, which "decorate" the function when it is called 
    # this is done when it is actively set.
    ###

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def animal_type(self):
        return self._animal_type # because of security only show 1st characters
    
    # dob property is returned as string, a string represents date outside object
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob setter, verifies date type before it is set or default to today
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "lbs" : self._lbs_food,
            "animal" : self._animal_type,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", lbs=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(lbs) > 0:
            self._lbs_food = int(lbs)
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return 
    
###
# db.Session.commit used to commit changes. 
# Wonder what would happen if that didn't happen ;)
# you can actually directly edit the object
# This would be an object oriented paradigm
###
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. User Object Constructors
    3. Try / Except 


In [95]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = Animal(name='Emaad', uid='emu', animal="emu", dob=datetime(1847, 2, 11), food=1000)
        u2 = Animal(name='Ryan', uid='mcwe', animal="bison", food=9000)
        u3 = Animal(name='Harsha', uid='harsh', animal="sloth", food=1000000)
        u4 = Animal(name='Lucas', uid='luke', animal="sparrow", food=4)
        u5 = Animal(name='Steven', uid='stevey', animal="Elephant", dob=datetime(1920, 10, 21), food=330)
        u6 = Animal(name='Varalu', uid='vlu', animal="Goose", dob=datetime(1921, 10, 21), food=10)
        # initializes classes and class variables. 
        # password not required, assigned a default value otherwise
        # in function definition

        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            #  intializes the stuff
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers() 
# actually calling function, tf breakpoint called first

Created new uid emu
Created new uid mcwe
Created new uid harsh
Created new uid luke
Created new uid stevey
Created new uid vlu


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
    2. user.password

In [96]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = Animal.query.filter_by(_uid=uid).first()
        # User is table, asks for query which is filtered by UID, and then it gets the first. 
    return user # returns user object



# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False # checks to see if everything works
    return False
        
#check_credentials("indi", "123qwerty")

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
    2. user = User(...)
    3. user.dob and try/except
    4. user.create() and try/except

In [97]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    nanimal = input("Enter animal type: ")
    lbs = input("Enter pounds of food to eat (per day in pounds): ")

    # Initialize User object before date
    user = Animal(name=name, 
                uid=uid, 
                animal=nanimal,
                food=lbs    
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Created
 {'id': 7, 'name': 'Fuck', 'uid': 'No', 'lbs': 12321428391208, 'animal': 'You', 'dob': '01-01-0101', 'age': 1922}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
    2. json_ready assignment, google List Comprehension

In [98]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = Animal.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Emaad',
  'uid': 'emu',
  'lbs': 1000,
  'animal': 'emu',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Ryan',
  'uid': 'mcwe',
  'lbs': 9000,
  'animal': 'bison',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 3,
  'name': 'Harsha',
  'uid': 'harsh',
  'lbs': 1000000,
  'animal': 'sloth',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 4,
  'name': 'Lucas',
  'uid': 'luke',
  'lbs': 4,
  'animal': 'sparrow',
  'dob': '03-15-2023',
  'age': 0},
 {'id': 5,
  'name': 'Steven',
  'uid': 'stevey',
  'lbs': 330,
  'animal': 'Elephant',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Varalu',
  'uid': 'vlu',
  'lbs': 10,
  'animal': 'Goose',
  'dob': '10-21-1921',
  'age': 101},
 {'id': 7,
  'name': 'Fuck',
  'uid': 'No',
  'lbs': 12321428391208,
  'animal': 'You',
  'dob': '01-01-0101',
  'age': 1922}]

In [100]:
def update():
    currid = input("ID of user to update: ")
    if currid == "":
        print("not updating!")
        return 
    user = find_by_uid(currid)
    print("user before: " + str(user))
    nUid = input("New UID? (Leave blank if no change): ")
    nName = input("New name? (Leave blank if no change): ")
    nFood = input("New food category? (Leave blank if no change): ")
    with app.app_context():
        user.update(nUid, nName, nFood)
    print("user after: " + str(user))

update()
    

not updating!


In [ ]:
def delete():
    uid = input("Username of person to delete: ")
    if uid == "":
        print("Not deleting anyone!")
        return
    user = find_by_uid(uid)
    with app.app_context():
        user.delete()
    print("Deleted!")
    return None
delete()

Deleted!


# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.